In [22]:
import pandas as pd
import numpy as np
import os
import datetime

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

pd.options.display.max_columns = 50

In [23]:
class DataFrameImporter:
    """
    Imports files in a directory and concats to a single dataframe
    """
    
    def __init__(self, path=None):
        self._df = self.search_directory(path)
        
    @property
    def df(self):
        return self._df
    @df.setter
    def df(self, value):
        self._df = value
        
    @staticmethod
    def search_directory(path=None):
        if path is not None:
            building_dfs = []
            for file in os.listdir(path):
                if file.endswith('xlsx'):
                    df = pd.read_excel(os.path.join(path, file), 
                                    na_values='-', 
                                    parse_dates=['Time'], 
                                    date_format='%Y-%m-%d %H:%M:%S')
                    building_dfs.append(df)
            return pd.concat(building_dfs)
        raise ValueError(f'Incorrect path. \tPath: {path}')


In [24]:
class ListSplitter:
    """
    Iterates through a list of integers. If difference between an integer and the next
    is more than 1, it will split the list.
    
    Returns a list of all split lists
    """
    
    @staticmethod
    def split_list_on_increment(lst):
        # get list of indices, if values in list increment by more than 1 split list
        sublists = []
        sublist = []
        for i in range(len(lst)):
            if i == 0 or lst[i] - lst[i-1] <= 1:
                sublist.append(lst[i])
            else:
                sublists.append(sublist)
                sublist = [lst[i]]
        sublists.append(sublist)
        # return all split lists (intervals)
        return sublists      

In [25]:
class DataFrameCleaner:
    """
    Implements basic cleaning functions to a DataFrame
    
    Can use an existing dataframe object, or import from path directory using DataFrame importer
    
    Specify building_no at instantiation as well
    """
    
    def __init__(self, df=None, path=None, building_no=None):
        if (df):
            self._df = df.sort_values()
        elif (path):
            self._df = DataFrameImporter(path).df
        else:
            raise ValueError("'df' or 'path' need to be specified")
        
        self.remap_columns()
        self.adjust_index()
        self.set_dtypes()
        self.add_time_columns()
        self.add_temp_columns()
        self.add_cumsum_columns()
        self.df['building_no'] = building_no
        
    @property
    def df(self):
        return self._df
    @df.setter
    def df(self, value):
        self._df = value
        
    def drop_dupplicates(self):
        self.df = self.df.drop_duplicates()
        
    def remap_columns(self):
        columns_map = {'Zone name':'Zone_name',
                    'Time':'Datetime',
                    'Zone temperature (degree Celsius)':'Zone_temp',
                    'Slab temperature (degree Celsius)': 'Slab_temp',
                    'Dew point temperature (degree Celsius)':'Dew_temp',
                    'Outside air temperature (degree Celsius)':'Ambient_temp',
                    'Damper status (%)':'Damper_status',
                    'Damper status':'Damper_status',
                    'Fan status':'Fan_status',
                    'Zone CO2 (ppm)':'Zone_c02',
                    'Louver status':'Louver_status'}
        self.df = self._df.rename(columns=columns_map)
    
    def adjust_index(self):
        self.df = self._df.sort_values(['Zone_name','Datetime']).reset_index(drop=True)

    def set_dtypes(self):
        dtypes_map = {'Zone_name':'object',
                    'Datetime':'datetime64[ns]',
                    'Zone_temp':float,
                    'Slab_temp':float,
                    'Dew_temp':float,
                    'Ambient_temp':float,
                    'Damper_status':float,
                    'Fan_status':'object',
                    'Zone_c02':float,
                    'Louver_status':'object'}
        
        for k,v in dtypes_map.items():
            if k in self.df.columns:
                self.df[k] = self.df[k].astype(v)
                
    def add_time_columns(self):
        # calc time between records
        self.df['Datetime_diff_mins'] = np.nan
        for (_, temp) in self.df.groupby(['Zone_name']):
            self.df.loc[temp.index,'Datetime_diff_mins'] = (temp.Datetime.diff()).dt.total_seconds() // 60

        self.df['Date'] = self.df.Datetime.dt.date
        self.df['Time'] = self.df.Datetime.dt.time
        self.df['Year'] = self.df.Datetime.dt.year
        self.df['Month'] = self.df.Datetime.dt.month
        self.df['Day'] = self.df.Datetime.dt.day
        self.df['DOW'] = self.df.Datetime.dt.dayofweek
        
        season_map = {12:1,1:1,2:1,3:2,4:2,5:2,6:3,7:3,8:3,9:4,10:4,11:4}
        self.df['Season'] = self.df.Month.map(season_map)
        

    def add_temp_columns(self):
        self.df['Zone_temp_diff'] = np.nan
        self.df['Slab_temp_diff'] = np.nan
        self.df['Dew_temp_diff'] = np.nan
        self.df['Ambient_temp_diff'] = np.nan
        
        for (_, temp) in self.df.groupby(['Zone_name']):
            self.df.loc[temp.index,'Zone_temp_diff'] = temp.Zone_temp.diff()
            self.df.loc[temp.index,'Slab_temp_diff'] = temp.Slab_temp.diff()
            self.df.loc[temp.index,'Dew_temp_diff'] = temp.Dew_temp.diff()
            self.df.loc[temp.index,'Ambient_temp_diff'] = temp.Ambient_temp.diff()
        
        
    def add_cumsum_columns(self):     
        # Fan status
        self.df['Cumulative_fan_on_mins'] = np.nan
        # self.df['Cumulative_fan_off_mins'] = np.nan
        self.df['Fan_on_group'] = np.nan
        
        # Damper status (0 to 100)
        if 'Damper_satus' in self.df.columns:
            self.df['Cumulative_damper_open_mins'] = np.nan
            self.df['Damper_open_group'] = np.nan
            
        # Louver = Close/Open
        if 'Louver_status' in self.df.columns:
            self.df['Cumulative_louver_open_mins'] = np.nan
            self.df['Louver_open_group'] = np.nan
        
        
        for (_, temp) in self.df.groupby(['Zone_name']):
            # calc cumulative time fan is on for each interval
            on_indices = ListSplitter.split_list_on_increment(temp[temp.Fan_status=='On'].index)
            for i, intervals in enumerate(on_indices):
                self.df.loc[intervals,'Cumulative_fan_on_mins'] = ((self.df.loc[intervals,'Datetime'].diff()).dt.total_seconds() // 60).cumsum()
                self.df.loc[intervals,'Fan_on_group'] = i+1
                
            # # calc cumulative time fan is off for each interval
            # off_indices = split_list_on_increment(temp[temp.Fan_status=='Off'].index)
            # for intervals in off_indices:
            #     self.df.loc[intervals,'Cumulative_fan_off_mins'] = ((self.df.loc[intervals,'Datetime'].diff()).dt.total_seconds() // 60).cumsum()
            
            if 'Damper_status' in self.df.columns:
                open_indices = ListSplitter.split_list_on_increment(temp[temp.Damper_status==100].index)
                for i, intervals in enumerate(open_indices):
                    self.df.loc[intervals,'Cumulative_damper_open_mins'] = ((self.df.loc[intervals,'Datetime'].diff()).dt.total_seconds() // 60).cumsum()
                    self.df.loc[intervals,'Damper_open_group'] = i+1
            
            if 'Louver_status' in self.df.columns:                  
                open_indices = ListSplitter.split_list_on_increment(temp[temp.Louver_status=='Open'].index)
                for i, intervals in enumerate(open_indices):
                    self.df.loc[intervals,'Cumulative_louver_open_mins'] = ((self.df.loc[intervals,'Datetime'].diff()).dt.total_seconds() // 60).cumsum()
                    self.df.loc[intervals,'Louver_open_group'] = i+1

#### For all Validation Checks: </br><b>Return True if FAULTY/INVALID</b>

In [26]:
# Temp:
# Faulty if any of the following are true:
    # Sudden drop in temp (typically to 0 or -1)
    # Value is out of range
    # Constant value for 24hrs+                   
class TemperatureValidation:
    """
    Class containing all temperature validation checks, works by specifying an area.
    
    eg area='Zone' will do all temperature validation checks for Zone_temp column
    """
    
    def __init__(self, df):
        self._df = df
        
    @property
    def df(self):
        return self._df
    
    def check_invalid_temperature_range(self, area, min=-5, max=50):
        # Return bool series of values that are not within valid temperature range
        return ~((self.df[f'{area}_temp'] > min) & (self.df[f'{area}_temp'] < max))
    
    def check_suboptimal_temperature_range(self, area, min=20, max=24):
        # Return bool series of values that are not within optimal temperature range
        return ~((self.df[f'{area}_temp'] > min) & (self.df[f'{area}_temp'] < max))
         
    def check_sudden_change(self, area, drop=15):
        # Return bool series of values that have not changed it in temp minimally
        return ~(abs(self.df[f'{area}_temp_diff']) < drop)
    
    def check_for_constant(self, area, hours=24):
        # Create bool series, set values to false
        temp_series = pd.Series([False for _ in range(len(self.df))], index=self.df.index)
        for (_, df) in self.df.groupby(['Zone_name']):
            constant_indices = ListSplitter.split_list_on_increment(df[df[f'{area}_temp_diff']==0].index)
            for indices in constant_indices:
                if len(indices)>1:
                    # total time = most recent time minus oldest time
                    total_time = (df.at[indices[-1],'Datetime'] - df.at[indices[0],'Datetime']).total_seconds() / 3600
                    if total_time >= hours:
                        # if total time is more than 24 hrs update all rows in between to True (invalid)
                        temp_series.loc[indices] = True
        return temp_series

In [27]:
# Zone co2:
    # Faulty if > 800ppm
class C02Validation:
    """
    Class containing C02 validation checks
    """
    
    def __init__(self, df):
        self._df = df
        
    @property
    def df(self):
        return self._df
        
    def check_invalid_ppm(self, max_ppm=800):
        return self.df.Zone_c02 > max_ppm

In [28]:
class OperatingHours:
    """
    Class containing building hour information
    
    Must set building_no when instantiating, can also optionally set amount of hours
    prior to open is the optimum start time for each building. default is 2
    """
    
    def __init__(self, building_no, optimum=2):
        self._df_oh = self.get_operating_hours(building_no, optimum)
        
    @property
    def df_oh(self):
        return self._df_oh
            
    @staticmethod
    def get_operating_hours(building_no, optimum):
        match building_no:
            case 1:
                oh_dict = {'open':[8,8,7,8,8,10,12], 'close':[21,21,21,21,21,17,17]}
            case 2:
                oh_dict = {'open':[8,8,8,8,8,8,8], 'close':[17,17,17,17,17,17,17]}
            case 3:
                oh_dict = {'open':[8,8,7,8,8,10,12], 'close':[18,18,18,18,18,18,18]}
            case _:
                print('Invalid building.\t Value: {building_no}')
                
        oh_dict['open'] = [datetime.time(hour) for hour in oh_dict['open']]
        oh_dict['close'] = [datetime.time(hour) for hour in oh_dict['close']]
        
        df = pd.DataFrame(oh_dict, index=[i for i in range (7)])
        df['optimum'] = (df.open.apply(lambda x: pd.to_datetime(x.strftime('%H:%M:%S'))) - datetime.timedelta(hours=optimum)).dt.time
        
        return df[['optimum','open','close']]
    
# Fan status:
# Faulty if:
    # Fan running anytime outside of operating hours, until optimum start time of building
    # Running when zone temp is within ideal range (20 to 24)
    # Running during occupied hrs when ambient temp < 18 or greater than 23 and zone co2 < 800 ppm and damper is fully open
class FanValidation:
    """
    Class containing fan validations
    
    Instantiate by specying building_no along with dataframe object
    """
    
    def __init__(self, df, building_no=None):
        if not (building_no):
            raise ValueError('building_no needs to be specified')
        self._df = df
        self._df_oh = OperatingHours(building_no).df_oh
        
    @property
    def df(self):
        return self._df
    
    @property
    def df_oh(self):
        return self._df_oh
    
    def check_fan_on_outside_oh(self, open_time='optimum'):
        # Get bool series of fan being on outside of operating hours up to open time (default=optimum)
        temp_series = pd.Series([False for _ in range(len(self.df))], index=self.df.index)
        
        # For each day of the week, check valid open and closing times
        for (_, df) in self.df[self.df.Fan_status=='On'].groupby('DOW'):
            # Get indices of when fan was on during closed hours
            indices = df[(df.Time>self.df_oh.at[df.DOW.unique()[0], 'close']) |\
                         (df.Time<self.df_oh.at[df.DOW.unique()[0], open_time])].index
            # Update bool series
            temp_series.loc[indices] = True
        return temp_series
    
    def check_fan_on_in_ideal_range(self, min=20, max=24):      
        # Get bool series of all fan on in ideal temp range
        return (self.df.Zone_temp>min)&(self.df.Zone_temp<max)&(self.df.Fan_status=='On')
    
    def check_fan_on_occupied_c02(self):
        # Get bool series where fan was on during occupied hours
        fan_on_during_occupied = pd.Series([False for _ in range(len(self.df))], index=self.df.index)
        for (_, df) in self.df[self.df.Fan_status=='On'].groupby('DOW'):
            indices = df[(df.Time<self.df_oh.at[df.DOW.unique()[0], 'close']) &\
                         (df.Time>self.df_oh.at[df.DOW.unique()[0], 'open'])].index
            fan_on_during_occupied.loc[indices] = True
            
        # Get bool series where temps were < 18 or > 23
        not_in_ideal_temperature_range = TemperatureValidation(self.df).check_suboptimal_temperature_range('Ambient', min=18, max=23)
        
        # Get bool series where co2 levels were < 800 ppm
        co2_levels_valid = ~C02Validation(self.df).check_invalid_ppm()
        
        # Get bool series where damper was fully open
        damper_fully_open = (self.df.Damper_status==100)
        
        # Return bool series of all conditions being met or not
        return (fan_on_during_occupied & not_in_ideal_temperature_range & co2_levels_valid & damper_fully_open)

In [64]:
class OscillationCounts:
    def count_oscillations_in_window(float_values):
        # Drop consecutive duplicates
        float_values = float_values.loc[float_values.shift() != float_values]
        
        # Identify transitions
        transitions = ((float_values.shift() == 0) & (float_values == 1)) | \
                    ((float_values.shift() == 1) & (float_values == 0))
        
        # Count transitions
        return transitions.sum()

class DamperValidation:
    """
    V1 Ccomplete
    """
    
    def __init__(self, df, building_no=None):
        if not (building_no):
            raise ValueError('building_no needs to be specified')
        self._df = df
        self._df_oh = OperatingHours(building_no).df_oh
        
    @property
    def df(self):
        return self._df
    
    @property
    def df_oh(self):
        return self._df_oh
    
    def check_damper_open_for_too_long(self, hours=24):
        damper_open_groups = self.df[self.df.Cumulative_damper_open_mins/60 >= hours].Damper_open_group.unique()
        return (self.df.Damper_open_group.isin(damper_open_groups))
    
    def check_damper_oscillation(self, window='6H', oscillations=6):
        # get minimum info necessary for finding invalid rows
        df_temp = self.df[['Datetime','Zone_name','Damper_status']]
        
        # create a boolean column setting to default value
        df_temp.insert(0,'Invalid', [False for _ in range(len(df_temp))])
        
        # iterate through each zone
        for (zone, df) in self.df.groupby('Zone_name'):
            
            temp = df[['Datetime','Damper_status']]
            
            temp = temp.set_index('Datetime').resample('10T').first().ffill()
            
            # Use a rolling window to apply the oscillation counting function
            temp['oscillations'] = temp['Damper_status'].rolling(window=window)\
                                .apply(OscillationCounts.count_oscillations_in_window, raw=False)
                                
            temp = temp.reset_index()

            sublists = ListSplitter.split_list_on_increment(temp[temp.oscillations>=oscillations].index)
            
            for sub in sublists:
                if (sub):
                    min_date = temp.loc[sub[0],'Datetime']
                    max_date = temp.loc[sub[-1],'Datetime']
                    df_temp.loc[(df_temp.Datetime>=min_date)&(df_temp.Datetime<=max_date)&(df_temp.Zone_name==zone),'Invalid'] = True
                    
        return df_temp.Invalid
            
                           
    
    
class LouverValidation:
    """
    V1 Complete
    """
    
    def __init__(self, df, building_no=None):
        if not (building_no):
            raise ValueError('building_no needs to be specified')
        self._df = df
        self._df_oh = OperatingHours(building_no).df_oh
        
    @property
    def df(self):
        return self._df
    
    @property
    def df_oh(self):
        return self._df_oh
        
    def check_louver_open_for_too_long(self, hours=24):
        louver_open_groups = self.df[self.df.Cumulative_louver_open_mins/60 >= hours].Damper_open_group.unique()
        return (self.df.Louver_open_group.isin(louver_open_groups))

    def check_louver_oscillation(self, window='6H', oscillations=6):
        # get minimum info necessary for finding invalid rows
        df_temp = self.df[['Datetime','Zone_name','Louver_status']]
        
        # create a boolean column setting to default value
        df_temp.insert(0,'Invalid', [False for _ in range(len(df_temp))])
        
        # iterate through each zone
        for (zone, df) in self.df.groupby('Zone_name'):
            
            temp = df[['Datetime','Damper_status','Louver_status']]
            temp.loc[:,'Louver_status'] = temp.Louver_status.apply(lambda x: 0 if x=='Close' else 1 if x=='Open' else np.nan).astype(float)
            
            temp = temp.set_index('Datetime').resample('10T').first().ffill()
            
            # Use a rolling window to apply the oscillation counting function
            temp['oscillations'] = temp['Louver_status'].rolling(window=window)\
                                .apply(OscillationCounts.count_oscillations_in_window, raw=False)
                                
            temp = temp.reset_index()

            sublists = ListSplitter.split_list_on_increment(temp[temp.oscillations>=oscillations].index)
            
            for sub in sublists:
                if (sub):
                    min_date = temp.loc[sub[0],'Datetime']
                    max_date = temp.loc[sub[-1],'Datetime']
                    df_temp.loc[(df_temp.Datetime>=min_date)&(df_temp.Datetime<=max_date)&(df_temp.Zone_name==zone),'Invalid'] = True
                    
        return df_temp.Invalid

    
    def check_louver_closed_occupied_valid_temp(self):
        # merge dataset with operating hours
        temp = self.df.merge(self.df_oh.reset_index().rename(columns={'index':'DOW'}), on='DOW')
        
        # conditions needed to find rows where louver is closed during operating hours
        cond1 = (temp.Louver_status=='Close')
        cond2 = ((temp.Datetime.dt.time>=temp.open)&(temp.Datetime.dt.time<temp.close))
        
        # conditions needed to find rows where ambient temp is between 18 and 23 degrees
        cond3 = ((temp.Ambient_temp>=18)&(temp.Ambient_temp<23))
        
        # conditions needed to find rows where dew point is below 19 degrees
        cond4 = (temp.Dew_temp<19)
        
        return (cond1&cond2&cond3&cond4)
    
    def check_louver_closed_occupied_diff_temps(self):
        # merge dataset with operating hours
        temp = self.df.merge(self.df_oh.reset_index().rename(columns={'index':'DOW'}), on='DOW')
        
        # conditions needed to find rows where louver is closed outside operating hours
        cond1 = (temp.Louver_status=='Close')
        cond2 = ((temp.Datetime.dt.time<temp.open)|(temp.Datetime.dt.time>=temp.close))
        
        # diff between ambient temp and zone temp greater than or equal to 3
        cond3 = ((temp.Ambient_temp-temp.Zone_temp).abs()>=3)
        
        # dew temp is less than 19
        cond4 = (temp.Dew_temp<19)
        
        return (cond1&cond2&cond3&cond4)
    

## Bringing all Validation Checks into a single class

In [30]:
class ValidationHandler:
    def __init__(self, df):
        all_series = []
        building_no = df.building_no.unique()[0]
        
        print(f'Building {building_no} validations started')
        
        valid_fan = FanValidation(df, building_no)
        all_series.append(valid_fan.check_fan_on_in_ideal_range())
        all_series.append(valid_fan.check_fan_on_outside_oh())
        
        if 'Zone_c02' in df.columns:
            valid_c02 = C02Validation(df)
            all_series.append(valid_c02.check_invalid_ppm())
            
            if 'Damper_status' in df.columns:
                all_series.append(valid_fan.check_fan_on_occupied_c02())
            else:
                print('Fan Validation skipped: check_fan_on_occupied_c02')
        else:
            print('C02 Validations skipped')
        
        valid_temp = TemperatureValidation(df)
        temp_keywords = ['Zone','Dew','Ambient','Slab']
        temp_functions = [valid_temp.check_for_constant, valid_temp.check_invalid_temperature_range, valid_temp.check_sudden_change]
        for word in temp_keywords:
            for function in temp_functions:
                all_series.append(function(word))
                
        if 'Damper_status' in df.columns:
            valid_damper = DamperValidation(df, building_no)
            all_series.append(valid_damper.check_damper_open_for_too_long())
            all_series.append(valid_damper.check_damper_oscillation())
        else:
            print('Damper status validations skipped')
            
        if 'Louver_status' in df.columns:
            valid_louver = LouverValidation(df, building_no)
            all_series.append(valid_louver.check_louver_open_for_too_long())
            all_series.append(valid_louver.check_louver_oscillation())
            all_series.append(valid_louver.check_louver_closed_occupied_valid_temp())
            all_series.append(valid_louver.check_louver_closed_occupied_diff_temps())
        else:
            print('Louver status validations skipped')
        
        valid_set = set()
        for series in all_series:
            temp_series = series[series].index
            if isinstance(temp_series, pd.RangeIndex):
                valid_set.update(temp_series.tolist())  # Convert RangeIndex to list before adding to set
        else:
            valid_set.update(temp_series)
        
        self._df = df.drop(valid_set).dropna(subset=['Fan_status'])
        
        print(f'Building {building_no} validations complete')
        
    @property
    def df(self):
        return self._df
    
    

## Importing and Concatting Building Data

In [31]:
test1 = DataFrameCleaner(path=r'D:\OneDrive - Swinburne University\Comp Sci\2024 Semester 1\Group Project\Data\exploratory_data\b1', building_no=1)

In [32]:
test2 = DataFrameCleaner(path=r'D:\OneDrive - Swinburne University\Comp Sci\2024 Semester 1\Group Project\Data\exploratory_data\b2', building_no=2)

In [33]:
test3 = DataFrameCleaner(path=r'D:\OneDrive - Swinburne University\Comp Sci\2024 Semester 1\Group Project\Data\exploratory_data\b3', building_no=3)

# Validated Datasets

In [54]:
# Building 1
df_valid1 = ValidationHandler(test1.df).df

Building 1 validations started
C02 Validations skipped
Louver status validations skipped
Building 1 validations complete


In [55]:
# Building 2
df_valid2 = ValidationHandler(test2.df).df

Building 2 validations started
Fan Validation skipped: check_fan_on_occupied_c02
Damper status validations skipped
Louver status validations skipped
Building 2 validations complete


In [65]:
# Building 3
df_valid3 = ValidationHandler(test3.df).df

Building 3 validations started


In [ ]:
# for i, df in enumerate([df_valid1, df_valid2, df_valid3]):
#     df.to_parquet(os.path.join(r'D:\OneDrive - Swinburne University\Comp Sci\2024 Semester 1\Group Project\Data\validated_data', f'building{i+1}.parquet'), index=False)

In [ ]:
test1.df['Faulty'] = True
test1.df.loc[df_valid1.index, 'Faulty'] = False

In [ ]:
test2.df['Faulty'] = True
test2.df.loc[df_valid2.index, 'Faulty'] = False

In [ ]:
test3.df['Faulty'] = True
test3.df.loc[df_valid3.index, 'Faulty'] = False

In [ ]:
for i, df in enumerate([test1.df, test2.df, test3.df]):
    df.to_parquet(os.path.join(r'D:\OneDrive - Swinburne University\Comp Sci\2024 Semester 1\Group Project\Data\cleaned_data', f'cleaned{i+1}.parquet'), index=False)

## Fan Status Value Counts

In [ ]:
print('Building 1')
display(df_valid1.Fan_status.value_counts(dropna=False))
display(df_valid1.groupby('Zone_name').Fan_status.value_counts())

print('Building 2')
display(df_valid2.Fan_status.value_counts(dropna=False))
display(df_valid2.groupby('Zone_name').Fan_status.value_counts())

print('Building 3')
display(df_valid3.Fan_status.value_counts(dropna=False))
display(df_valid3.groupby('Zone_name').Fan_status.value_counts())

Building 1


Fan_status
On     779635
Off    278873
Name: count, dtype: int64

Zone_name  Fan_status
Acu101     On            34462
           Off           13652
Acu102     On            34462
           Off           13652
Acu103     On            34462
           Off           13652
Acu104     On            33964
           Off           14150
Acu105     On            33902
           Off           14212
Acu106     On            33899
           Off           14215
Acu107     On            42058
           Off            6056
Acu201     On            35664
           Off           12450
Acu202     On            35314
           Off           12800
Acu203     On            35664
           Off           12450
Acu204     On            35664
           Off           12450
Acu205     On            35664
           Off           12450
Acu206     On            35664
           Off           12450
Acu207     On            35664
           Off           12450
Acu301     On            35391
           Off           12723
Acu302     On            35391
           Off   

Building 2


Fan_status
Off    119852
On      17938
Name: count, dtype: int64

Zone_name  Fan_status
Ac-2-1     Off           22502
           On             5056
Ac-2-2     Off           27190
           On              368
Ac-2-3     Off           26071
           On             1487
Ac-2-4     Off           21422
           On             6136
Ac-2-5     Off           22667
           On             4891
Name: count, dtype: int64

Building 3


Fan_status
Off    467525
On     152400
Name: count, dtype: int64

Zone_name  Fan_status
G-01       Off           38710
           On             2618
G-02       Off           36099
           On             5230
G-03       Off           31462
           On             9866
G-04       Off           34749
           On             6579
G-05       Off           38296
           On             3033
G-06       Off           31039
           On            10289
G-07       Off           32738
           On             8590
G-08       Off           30790
           On            10538
L1-01      On            22637
           Off           18691
L1-02      Off           32750
           On             8579
L1-03      On            41329
L1-04      Off           26572
           On            14757
L1-05      Off           32979
           On             8349
L1-06      Off           41328
L1-07      Off           41322
           On                6
Name: count, dtype: int64